# Module

In [8]:
import warnings
warnings.filterwarnings(action='ignore')
import os
import sys
import dotenv
import nest_asyncio
dotenv.load_dotenv()
import requests
import json
from glob import glob
import time
import pandas as pd
from tqdm import tqdm
from urllib.parse import unquote
from collections import defaultdict

# Setting

In [9]:
api_key = api_key = os.getenv("UPSTAGE_API_KEY")
file_name_list = glob("../data/nursing/*.pdf")
headers = {"Authorization": f"Bearer {api_key}"}
document_parser_url = "https://api.upstage.ai/v1/document-ai/async/document-parse"
document_parser_url_ = "https://api.upstage.ai/v1/document-ai/document-parse"
check_url = "https://api.upstage.ai/v1/document-ai/requests"


In [19]:
files = {"document": open(file_name, "rb")}
data = {
    "ocr": "force",
    "output_formats": "['markdown', 'html', 'text']"
    }
api_response = requests.post("https://api.upstage.ai/v1/document-ai/document-parse", headers=headers, files=files, data=data)

In [20]:
api_response.json()

{'api': '2.0',
 'content': {'html': '<header id=\'0\' style=\'font-size:18px\'>CHAPTER 14 호흡기장애 대상자 간호 I 553</header>\n<br><figure><img id=\'1\' alt="" data-coord="top-left:(1144,42); bottom-right:(1318,155)" /></figure>\n<h1 id=\'2\' style=\'font-size:20px\'>표 14-15 폐렴 대상자 간호</h1>\n<br><table id=\'3\' style=\'font-size:14px\'><tr><td>기대되는 결과</td><td>간호</td></tr><tr><td>비효과적인 호흡양상 개선 · 효과적으로 기침하고 호흡함 · 물을 3 L/일 마심 · 활력징후 정상 · 청진 시 폐음 정상 · 지시대로 물을 마심 · 객담을 잘 뱉어냄 · 소변색이 정상임</td><td>· 활력징후와 신체상태를 관찰한다. · 호흡의 횟수, 리듬, 깊이, 노력을 관찰한다. · 흉부의 움직임과 대칭 여부, 보조근의 사용, 쇄골 움직임, 늑간근 수축을 관찰한다. · 호흡양상을 관찰한다. 빈호흡, 빈맥, 과도환기, Kussmaul 호흡, Cheyne-Stokes 호흡, 지속성 흡식 (apneustic) 호흡, Biot 호흡, 실조성(ataxic) 호흡양상 등 · 횡격막의 근육피로를 관찰한다(paradoxical motion). · 호흡음을 청진하여 환기 감소나 결핍부위, 진통음(adventitions)이 틀리는 부위를 기록한다. · 악설음과 천명음을 청진하여 흡인이 필요한지 결정한다. · 동맥혈가스분석 결과를 확인하고, 안절부절못함, 불안 등의 산소부족 증상을 관찰한다. · 효과적인 기침 능력을 관찰한다: 기침의 시작, 특징, 기간 · 호흡기 분비물을 관찰한다. · 호흡곤란을 완화시키거나 악화시키는 요인을 확인한다. · 흉부 X-선 결과를 확인한다. · 심호흡으로 흉곽내 압력을 상승시켜 강하게 숨을

In [14]:
files = {"document": open(file_name, "rb")}
data = {
    "ocr": "force",
    "output_formats": "['markdown', 'html', 'text']"
    }
api_response = requests.post("https://api.upstage.ai/v1/document-ai/async/document-parse", headers=headers, files=files, data=data)

In [16]:
api_response.json()

{'request_id': '20e367d4-6578-4a87-9667-7826b2dcd93e'}

In [17]:
a = requests.get(os.path.join(check_url, "20e367d4-6578-4a87-9667-7826b2dcd93e"), headers=headers)

In [18]:
a.json()

{'id': '20e367d4-6578-4a87-9667-7826b2dcd93e',
 'status': 'completed',
 'model': 'document-parse',
 'failure_message': '',
 'total_pages': 3,
 'completed_pages': 3,
 'batches': [{'id': 0,
   'model': 'document-parse-240910',
   'status': 'completed',
   'failure_message': '',
   'download_url': 'https://api.upstage.ai/v1/files/images/ba7b7fd0-cddd-4f7b-842d-4ac5a920d239/20/e3/67/nursing_test.pdf_batch0000_0001_0003.json?Expires=1729066734&Signature=hEbWLh-98Nm~MXzY0kiLQQ6rNjibTQWlDyiMjhVKt9Fd76T7eudW3OiTtFRMg7qw40sj32nRk82TBLO9dW5Y8Z9il4ialD21tbzC77kTe1rWoedmT6qKQhEEZbActm-DFlLZkSNZFpUgm~N8RKfttb-PmuxsL9MsPc1bkjkLDEmC5lk7ZvV8xAm48bEbUUI9wfKzaKQo-15k4pDYIH~h-QP39JlxmzFlFoHvaYGCqWzLeg-tM8OrIRuEV8f-GP8z4wQ9BEffrtP0D6KJmXpAuQV-HD41V4kfTkP8XZ6st8wga-eQhG-d3JFe0jzzuLcSckF80O7MIIAAwGSflBjvyw__&Key-Pair-Id=KHHXCBDYXSZX3',
   'start_page': 1,
   'end_page': 3,
   'requested_at': '2024-10-16T08:03:20Z',
   'updated_at': '2024-10-16T08:03:30Z'}],
 'requested_at': '2024-10-16T08:03:20Z',
 'complet

# Parsing

In [13]:
# document_file_name,request_id, download_url 이 컬럼인 df를 만들어서 csv로 저장 하는 로직 구현
df = []
for file_name in tqdm(file_name_list):
    try:
        print(f"{file_name} - Start!")
        files = {"document": open(file_name, "rb")}
        data = {
            "ocr": "force",
            "output_formats": '["markdown", "html", "text"]'
            }
        api_response = requests.post(document_parser_url, headers=headers, files=files, data=data)
        while True:
            check_response = requests.get(os.path.join(check_url, api_response.json()["request_id"]), headers=headers)
            is_completed = False
            for batch in check_response.json()["batches"]:
                if batch["status"] == "completed":
                    is_completed = True
                else:
                    is_completed = False
                    break
            if is_completed:
                print(f"{file_name} - Completed!")
                break
            else:
                time.sleep(10)
        for batch in check_response.json()["batches"]:
            df.append([file_name, api_response.json()["request_id"], batch])
        print(f"==============================================")
    except Exception as e:
        print(f"{file_name} - error")
        print(e)
        continue

# df를 csv로 저장
df = pd.DataFrame(df, columns=["document_file_name", "request_id", "batch"])
df.to_csv("../data/nursing/parse_result_test.csv", index=False)

  0%|          | 0/1 [00:00<?, ?it/s]

/home/livin/rag_pipeline/AutoRAG/data/test/nursing_test.pdf - Start!


100%|██████████| 1/1 [00:20<00:00, 20.61s/it]

/home/livin/rag_pipeline/AutoRAG/data/test/nursing_test.pdf - Completed!


# Check Result

##### Check request_id

In [8]:
check_response = requests.get(check_url, headers=headers).json()["requests"]

##### Downloading 함수 정의

In [25]:
def download_file(url, folder_path):
    # 폴더가 존재하지 않으면 생성
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    # 파일 다운로드 시작
    response = requests.get(url, stream=True)
    response.raise_for_status()  # 다운로드가 실패했는지 확인

    file_name = url.split("/")[-1]
    # 파일명을 한글 인코딩을 고려하여 디코딩
    file_name = unquote(file_name).split("?")[0]


    # 저장 경로 지정
    file_path = os.path.join(folder_path, file_name)

    # 파일 저장
    with open(file_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)
    print(f"File saved at {file_path}")


##### Downloading

In [ ]:
folder_path = "./downloads"

for check_response_i in check_response:
    if check_response_i["status"] == "completed":
        batch_results = requests.get(os.path.join(check_url, check_response_i["id"]), headers=headers).json()["batches"]
        for batch_result in batch_results:
            download_file(batch_result["download_url"], folder_path)
        print("Completed")
    else:
        print("Not completed yet")
        continue

# File Integration

##### html 파일로 저장

In [3]:
json_file_list = sorted(glob("../data/nursing/nursing_downloads/*"))
file_names = sorted(list(set([name.split(".pdf")[0].split("/")[-1] for name in name_list])))
file_name_to_json = defaultdict(list)

for file_name in file_names:
    for json_file in json_file_list:
        if file_name in json_file:
            file_name_to_json[file_name].append(json_file)

NameError: name 'name_list' is not defined

In [41]:
for file_name in file_name_to_json:
    file_corpus = ""
    for json_file in file_name_to_json[file_name]:
        with open(json_file, "r") as f:
            data_elements = json.load(f)
            for element in data_elements["elements"]:
                file_corpus += element["html"]
    # html 파일로 저장
    with open(f"../data/nursing/nursing_html/{file_name}.html", "w") as f:
        f.write(file_corpus)

##### corpus 파일로 변환 후 저장

In [54]:
df_dict = {"texts":[], "path":[], "page":[], "last_modified_datetime":[]}

In [55]:
for file_name in file_names:
    with open(f"../data/nursing/nursing_html/{file_name}.html", "r") as f:
        html = f.read()
        df_dict["texts"].append(html)
        df_dict["path"].append(file_name)
        df_dict["page"].append(-1)
        df_dict["last_modified_datetime"].append(pd.Timestamp.now())

In [56]:
df = pd.DataFrame(df_dict)

In [58]:
# csv 파일로 저장   
df.to_csv("../data/nursing/corpus/nursing_corpus.csv")

# parquet 파일로 저장   
df.to_parquet("../data/nursing/corpus/nursing_corpus.parquet")

In [6]:
# json 모듈로 파일을 읽기

import json
with open("/home/livin/rag_pipeline/AutoRAG/data/nursing/nursing_downloads/23장 내분비장애 대상자 간호.pdf_batch0002_0021_0030.json", "r") as f:
    data = json.load(f)

In [7]:
data

{'api': '1.1',
 'billed_pages': 10,
 'elements': [{'bounding_box': [{'x': 337, 'y': 116},
    {'x': 600, 'y': 116},
    {'x': 600, 'y': 142},
    {'x': 337, 'y': 142}],
   'category': 'header',
   'html': "<header id='0' style='font-size:16px'>UNIT 7 조정장애 대상자 간호</header>",
   'id': 0,
   'page': 21,
   'text': 'UNIT 7 조정장애 대상자 간호'},
  {'bounding_box': [{'x': 249, 'y': 116},
    {'x': 331, 'y': 116},
    {'x': 331, 'y': 143},
    {'x': 249, 'y': 143}],
   'category': 'paragraph',
   'html': "<br><p id='1' data-category='paragraph' style='font-size:18px'>566 I</p>",
   'id': 1,
   'page': 21,
   'text': '566 I'},
  {'bounding_box': [{'x': 185, 'y': 220},
    {'x': 416, 'y': 220},
    {'x': 416, 'y': 247},
    {'x': 185, 'y': 247}],
   'category': 'paragraph',
   'html': "<p id='2' data-category='paragraph' style='font-size:18px'>표 23-3 요붕증 치료약물</p>",
   'id': 2,
   'page': 21,
   'text': '표 23-3 요붕증 치료약물'},
  {'bounding_box': [{'x': 172, 'y': 235},
    {'x': 1186, 'y': 235},
    {'x': 11